In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv('../ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

In [3]:
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movies = movies[['item_id', 'movie title']]

In [5]:
movies.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
movies['item_id'] = movies['item_id'] - 1

In [7]:
movies.head()

,item_id,movie title
0,0,Toy Story (1995)
1,1,GoldenEye (1995)
2,2,Four Rooms (1995)
3,3,Get Shorty (1995)
4,4,Copycat (1995)


In [12]:
user_list = ratings['user_id'].unique()

In [13]:
user_list = np.array(sorted(user_list))

In [14]:
user_list

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [11]:
K = 20
lr = 1e-4
batch_size = 128
epochs = 100

In [60]:
class MF(tf.keras.Model):
    def __init__(self, n_users, n_items, K=20):
        super(MF, self).__init__()

        self.u_embedding = tf.keras.layers.Embedding(n_users, K, name="user_embedding")
        self.i_embedding = tf.keras.layers.Embedding(n_items, K, name="item_embedding")
        self.flatten = tf.keras.layers.Flatten()
        self.dot = tf.keras.layers.Dot(axes=2)

    def call(self, input, training=False):
        u = self.u_embedding(input[0])
        i = self.i_embedding(input[1])

        x = self.dot([u, i])

        return x

    def train_step(self, data):
        input, y = data

        with tf.GradientTape() as tape:
            y_pred = self(input, training=True)

            self.indices = tf.sparse.from_dense(y).indices # y에서 0이 아닌부분만의 index들

            loss = self.compiled_loss(tf.gather_nd(y, self.indices), tf.gather_nd(y_pred, self.indices),  regularization_losses=self.losses) # loss 값 업데이트는, 위의 indices들에 대해서만 해야함.

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name : m.result() for m in self.metrics}

In [61]:
y = tf.constant([[[0, 2, 3, 0],
                 [3, 5, 0, 0],
                 [1, 3, 2, 4],
                 [5, 0, 1, 2]]])
y_pred = tf.constant([[[0, 2.1, 2.8, 0],
                 [3, 4.7, 0, 0],
                [0.8, 2.5, 1.9, 3.4],
                [4.7, 0, 0.8, 1.9]]])

In [63]:
model = MF(4, 4, K = 3)

In [64]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=2e-2, momentum=0.9),
              loss="mse",
              metrics=["mse"])

In [65]:
user = np.expand_dims(np.arange(4), axis=0)
movie = np.expand_dims(np.arange(4), axis=0)

In [66]:
model.fit([user, movie], y, epochs=150)

Epoch 1/150
1/1 [==============================] - 1s 651ms/step - loss: 9.7246 - mse: 6.6856
Epoch 2/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7242 - mse: 6.6854
Epoch 3/150
1/1 [==============================] - 0s 0s/step - loss: 9.7235 - mse: 6.6849
Epoch 4/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7224 - mse: 6.6841
Epoch 5/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7209 - mse: 6.6831
Epoch 6/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7191 - mse: 6.6819
Epoch 7/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7167 - mse: 6.6803
Epoch 8/150
1/1 [==============================] - 0s 0s/step - loss: 9.7138 - mse: 6.6782
Epoch 9/150
1/1 [==============================] - 0s 0s/step - loss: 9.7102 - mse: 6.6757
Epoch 10/150
1/1 [==============================] - 0s 10ms/step - loss: 9.7056 - mse: 6.6726
Epoch 11/150
1/1 [==============================] - 0s 0s/step - loss: 9.6

In [67]:
model.predict([user, movie])

1/1 [==============================] - 0s 90ms/step


array([[[-0.7090238,  2.06649  ,  2.9199297,  3.8746405],
        [ 2.9660711,  5.0333786,  4.610376 ,  6.9422174],
        [ 1.0552   ,  2.840984 ,  2.109292 ,  4.0154457],
        [ 4.9955735,  2.7059636,  0.9632154,  2.0077744]]], dtype=float32)

## 여기서 부터 다시 짜라.. 22.10.08
- <s>loss 짜야하는데, MF는 y value에서 0이 아닌값, 즉 관측된 값을 가지고 y_pred와 MSE를 계산해줘야함</s> -- 22.10.09 해결
- Metric에서의 MSE 계산도 y value의 0이 아닌 인덱스에 대해서 해야할까... 22.10.09